In [40]:
from pathlib import Path
from sampling_mining_workflows_dsl.operator.OperatorBuilder import OperatorBuilder
from sampling_mining_workflows_dsl.WorkflowBuilder import WorkflowBuilder

from sampling_mining_workflows_dsl.element.Loader import Loader
from datetime import datetime
from sampling_mining_workflows_dsl.metadata.Metadata import Metadata
from sampling_mining_workflows_dsl.element.writer.JsonWriter import JsonWriter
from sampling_mining_workflows_dsl.element.writer.CsvWriter import CsvWriter

from sampling_mining_workflows_dsl.swh.loader import *
from sampling_mining_workflows_dsl.Workflow import Workflow

from sampling_mining_workflows_dsl.analysis.HistWorkflowAnalysis import (
    HistWorkflowAnalysis,
)
from sampling_mining_workflows_dsl.analysis.KSWorkflowAnalysis import KSWorkflowAnalysis
from sampling_mining_workflows_dsl.exec_visualizer.WorkflowVisualizer import WorkflowVisualizer




In [44]:
import importlib
from sampling_mining_workflows_dsl.analysis import HistWorkflowAnalysis
from sampling_mining_workflows_dsl.analysis import HistAnalysis
from sampling_mining_workflows_dsl.analysis import KSWorkflowAnalysis



importlib.reload(HistAnalysis)

importlib.reload(HistWorkflowAnalysis)
importlib.reload(KSWorkflowAnalysis)
from sampling_mining_workflows_dsl.analysis.KSWorkflowAnalysis import KSWorkflowAnalysis
from sampling_mining_workflows_dsl.analysis import HistAnalysis
from sampling_mining_workflows_dsl.analysis.HistWorkflowAnalysis import HistWorkflowAnalysis

In [3]:

workflow = WorkflowBuilder()\
    .input(SwhLoader(swh_id,swh_commit_count,swh_commiter_count,swh_latest_commit_date))\
    .filter_operator(swh_latest_commit_date.is_after(datetime(2023, 1, 1)))\
    .grouping_operator(
        # First stratum: projects with less than 5 authors
        (
            OperatorBuilder(Workflow())
            .filter_operator("swh_commiter_count < 5")
            .random_selection_operator(10000,seed=42)
        ),
        # Second stratum: projects with 5 or more authors
        (
            OperatorBuilder(Workflow())
            .filter_operator("swh_commiter_count >=5")
            .random_selection_operator(10000, seed=42)
        ),
    )\
    .union_operator()\
    .output(JsonWriter("result/sampled_repos.json"))\
    .execute_workflow()\
    .print()
    



Loading SWH repositories...


Filtering elements: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 27133980/27133980 [29:57<00:00, 15099.42repo_id/s]


SWH repositories loaded.
cache
SWH repositories cached.


Filtering elements: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 650395/650395 [00:05<00:00, 114604.55element/s]


Writing set to JSON at: result/sampled_repos.json
JSON has been written to result/sampled_repos.json
Workflow: [[[
FilterOperator
    
    input:
            (size=27133980)[213243331,4750636176,2600672795,4335747694,7296573595,1203459508,975121593,1244712943,3897451982,1474439158...]
    output:
            (size=650395)[1158900716,3575202660,6047223259,7868358693,1120553623,6092286405,6318978799,370924387,6239234108,7553547400...]
    nextOperator:
            GroupingOperator
                
                Internal Workflows:
                Workflow: [[[
                FilterOperator
                    
                    input:
                                            (size=650395)[1158900716,3575202660,6047223259,7868358693,1120553623,6092286405,6318978799,370924387,6239234108,7553547400...]
                    output:
                                            (size=475832)[1158900716,7868358693,1120553623,6318978799,370924387,7553547400,4042855135,153013930,6080723676,

In [34]:
HistWorkflowAnalysis(metadata=swh_commit_count,top_x=-1,category=False,sort=True,output_path="result/hist/out_truncated",log_y=True,max_x_bound=7000000,x_label="Number of Commits",fixed_bins=20,fig_size=(9,4)).analyze(workflow)


In [ ]:
HistWorkflowAnalysis(metadata=swh_commit_count,top_x=-1,category=False,sort=True,output_path="result/hist/out_without_limit",log_y=True,fixed_bins=20,x_label="Number of Commits").analyze(workflow)


In [45]:
WorkflowVisualizer(workflow,"result/workflow_visualization").generate_graph()
KSWorkflowAnalysis(metadata=swh_commit_count,output_path="result").analyze(workflow)

Workflow graph saved to result/workflow_visualization/workflow_graph
HTML page saved to result/workflow_visualization/workflow.html
KS Analysis results will be saved in result/ks_analysis.txt
Getting all sets from workflow...
Getting all sets from workflow...
Getting all sets from workflow...
Analyzing sets 0 from Set #0Initial set and 0 from Set #0Initial Input
  [PASSED] KS Statistic: 0.0, P-value: 1.0
Analyzing sets 0 from Set #0Initial set and 1 from Set #1FilterOperator
  [FAILED] KS Statistic: 0.2146415002327809, P-value: 0.0
Analyzing sets 0 from Set #0Initial set and 2 from Set #2FilterOperator
  [FAILED] KS Statistic: 0.1216652520522074, P-value: 0.0
Analyzing sets 0 from Set #0Initial set and 3 from Set #3RandomSelectionOperator
  [FAILED] KS Statistic: 0.11988867471708897, P-value: 1.1812932922711352e-125
Analyzing sets 0 from Set #0Initial set and 4 from Set #4FilterOperator
  [FAILED] KS Statistic: 0.5950494278648907, P-value: 0.0
Analyzing sets 0 from Set #0Initial set an

{'passing_pairs': [{'comparison': 1,
   'set_1': 'Set #0Initial set',
   'set_2': 'Set #0Initial Input',
   'statistic': np.float64(0.0),
   'p_value': np.float64(1.0),
   'interpretation': 'No significant difference (p > 0.05)'},
  {'comparison': 8,
   'set_1': 'Set #1FilterOperator',
   'set_2': 'Set #1FilterOperator',
   'statistic': np.float64(0.0),
   'p_value': np.float64(1.0),
   'interpretation': 'No significant difference (p > 0.05)'},
  {'comparison': 14,
   'set_1': 'Set #2FilterOperator',
   'set_2': 'Set #2FilterOperator',
   'statistic': np.float64(0.0),
   'p_value': np.float64(1.0),
   'interpretation': 'No significant difference (p > 0.05)'},
  {'comparison': 15,
   'set_1': 'Set #2FilterOperator',
   'set_2': 'Set #3RandomSelectionOperator',
   'statistic': np.float64(0.006308712318633503),
   'p_value': np.float64(0.8280814365146424),
   'interpretation': 'No significant difference (p > 0.05)'},
  {'comparison': 19,
   'set_1': 'Set #3RandomSelectionOperator',
   'se